In [1]:
import pandas as pd

DATA = '/kaggle/input/nuclear-share-of-electricity-generation/Nuclear_Electricity_Statistics_2022.csv'
FIXES = {'IRAN, ISLAMIC REPUBLIC OF': 'IRAN', 'NETHERLANDS, KINGDOM OF THE': 'NETHERLANDS', 'UNITED STATES OF AMERICA': 'UNITED STATES',}
ISO = '/kaggle/input/country-mapping-iso-continent-region/continents2.csv'

df = pd.read_csv(filepath_or_buffer=DATA)

# drop the total row
df = df[df['Country'] != 'Total']
# apply some name fixes so we can join with the ISO data
df['Country'] = df['Country'].apply(func=lambda x: FIXES[x] if x in FIXES.keys()  else x)
# get the ISO data
iso_df = pd.read_csv(filepath_or_buffer=ISO, usecols=['alpha-3', 'name'])
# build our join key column
iso_df['Country'] = iso_df['name'].str.upper()
# do the merge and clean up
df = df.merge(right=iso_df, on='Country').drop(columns=['Country'])
df.head()

,Total Net Electrical Capacity [MW],Number of Operated Reactors,Nuclear Electricity Supplied [GW.h],Nuclear Share [%],name,alpha-3
0,1641,3,7469.52,5.4,Argentina,ARG
1,416,1,2630.85,31.0,Armenia,ARM
2,1110,1,4411.35,11.9,Belarus,BLR
3,5942,7,41744.41,46.4,Belgium,BEL
4,1884,2,13744.82,2.5,Brazil,BRA


Now we're ready to plot. 

In [2]:
from plotly import express
for color in ['Total Net Electrical Capacity [MW]', 'Number of Operated Reactors', 'Nuclear Electricity Supplied [GW.h]', 'Nuclear Share [%]',]:
    express.choropleth(data_frame=df, locations='alpha-3', color=color, title=color).show()

For fun let's look at correlations among our numerical columns.

In [3]:
express.imshow(img=df.corr(numeric_only=True))

Clearly the first three columns are essentially the same data, while the last column has another variable embedded in it, that being the total energy generated in each country.